# Snowpark For Python -- Advertising Spend and ROI Prediction

### Objective

In this session, we will train a Linear Regression model to predict future ROI (Return On Investment) of variable ad spend budgets across multiple channels including search, video, social media, and email using Snowpark for Python and scikit-learn. 

In this Notebook, we will:

* Create a Session object and securely connect to Snowflake
* Load data from Snowflake table into Snowpark DataFrame
* Perform Exploratory Data Analysis (EDA) on Snowpark DataFrame
* Pivot and Join datasets
* Create a Python Stored Procedure to deploy model training code on Snowflake
* Create Python Scalar and Vectorized User-Defined Functions (UDF) for inference on new data
* Create a Snowflake Task to automate (re)training of the model


### Instructions

This workshop is meant to be hands-on and we've built in exercises throughout the notebook. Whenever you see **YOUR TURN**, this is your queue to complete the following exercise. If you get stuck, the solutions can be found in the *Snowpark_For_Python_Solution.ipynb* notebook in the same repository.

Additionally, when we create the **stored procedure**, **UDF**, and **Task** for model training and scoring, you'll need to add your initials or a unique identifier to the name. This is because we are using the same database and schema and don't want your functions overwriting eachother. There are reminders below as well when you get to that part of the code.

### Prerequisites

  - Log into snowflake account and switch to ACCOUNTADMIN role
    - Click on the **Billing** on the left side panel
    - Click on [Terms and Billing](https://app.snowflake.com/terms-and-billing)
    - Read and accept terms to continue with the workshop
  - Python 3.8
  - Create and Activate Conda Environment (OR, use any other Python environment with Python 3.8) 
    - conda create --name snowpark -c https://repo.anaconda.com/pkgs/snowflake python=3.8
    - conda activate snowpark
  - Install Snowpark for Python, Streamlit and other libraries in Conda environment
    - conda install -c https://repo.anaconda.com/pkgs/snowflake snowflake-snowpark-python pandas notebook scikit-learn cachetools nbformat plotly
  - Update [connection.json](connection.json) with your Snowflake account details and credentials

<div style='text-align: center'>
    <img src="assets/snowpark.png" alt="Snowpark" style="width: 75%;"/>
</div>

### Import Libraries

In [1]:
# Snowpark for Python
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.functions import col

# Misc
import json
import sys
import logging

import numpy as np
import plotly.express as px

logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

### Establish Secure Connection to Snowflake

Using the Snowpark API, it’s quick and easy to establish a secure connection between Snowflake and Notebook.

 *Connection options: Username/Password, MFA, OAuth, Okta, SSO*

In [2]:
#Test Code
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
#session.sql_simplifier_enabled = True

session.sql('use role snowpark_workshop_role').collect()
session.sql('use warehouse wh_snowpark_hol').collect()
#session.sql('use database DB_SNOWPARK_HOL').collect()
#session.sql('use SCHEMA ROI_PREDICTION').collect()

snowflake_environment = session.sql(
    '''select
        current_user()
        ,current_role()
        ,current_database()
        ,current_schema()
        ,current_version()
        ,current_warehouse()'''
    ).collect()

snowpark_version = VERSION

# Current Environment Details
print(f'User: {snowflake_environment[0][0]}')
print(f'Role: {snowflake_environment[0][1]}')
print(f"Database: {snowflake_environment[0][2]}")
print(f'Snowflake version: {snowflake_environment[0][4]}')
print(f"Schema: {snowflake_environment[0][3]}")
print(f"Warehouse: {snowflake_environment[0][5]}")
print(f'Snowpark for Python version: {snowpark_version[0]}.{snowpark_version[1]}.{snowpark_version[2]}')



User: RECKER_SFC
Role: SNOWPARK_WORKSHOP_ROLE
Database: DB_SNOWPARK_HOL
Snowflake version: 7.37.2
Schema: ROI_PREDICTION
Warehouse: WH_SNOWPARK_HOL
Snowpark for Python version: 1.4.0


### Load Aggregated Campaign Spend Data from Snowflake table into Snowpark DataFrame

Let's first load the campaign spend data. This table contains ad click data that has been aggregated to show daily spend across digital ad channels including search engines, social media, email and video.

NOTE: Ways to load data in a Snowpark Dataframe
* ```session.table("db.schema.table")```
* ```session.sql("select col1, col2... from tableName")```
* ```session.read.parquet("@stageName/path/to/file")```
* ```session.create_dataframe([1,2,3], schema=["col1"])```

TIP: For more information on Snowpark DataFrames, refer to the [docs](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/_autosummary/snowflake.snowpark.html#snowflake.snowpark.DataFrame).


In [3]:
snow_df_spend = session.table('DAILY_SPEND')



<div style='text-align: center'>
    <img src="assets/snowpark_python_api.png" alt="Snowpark" style="width: 75%;"/>
</div>

In [4]:
# Action sends the DF SQL for execution
# Note: history object provides the query ID which can be helpful for debugging as well as the SQL query executed on the server
with session.query_history() as history:
    snow_df_spend.show(30)

history.queries

---------------------------------------------------------------
|"CHANNEL"          |"DATE"      |"DAILYCOST"  |"DAILYLEADS"  |
---------------------------------------------------------------
|SEM                |2021-08-21  |37158        |340           |
|3rd Party Listing  |2021-08-21  |27685        |650           |
|Social             |2021-08-22  |0            |0             |
|SEM                |2021-08-22  |0            |0             |
|3rd Party Listing  |2021-08-22  |0            |0             |
|Manually Entered   |2021-08-22  |0            |0             |
|Display            |2021-08-22  |0            |0             |
|Direct/Organic     |2021-08-22  |0            |0             |
|Email              |2021-08-22  |0            |0             |
|Radio              |2021-08-22  |0            |0             |
|Alloy              |2021-08-22  |0            |0             |
|Digital Not Set    |2021-08-22  |0            |0             |
|Direct/Organic     |2021-08-23  |0     

[QueryRecord(query_id='01afdcff-0001-6fd6-0003-b7d2000d22ca', sql_text='SELECT  *  FROM DAILY_SPEND LIMIT 30')]

### Pandas vs Snowpark DataFrame memory comparison

Snowpark and Pandas DataFrame are similar but do have some notable differences.

One of the biggest differences is the amount of local memory consumed. Run the cell block below to see firsthand.

In [5]:
# get the size in MB of a dataframe
def get_df_memory_size(df) -> float:
    return np.round(sys.getsizeof(df) / (1024.0**2), 2)

pandas_df_spend = snow_df_spend.to_pandas()

print(f'Size in MB of Snowpark DataFrame in Memory: {get_df_memory_size(snow_df_spend)}')
print(f'Size in MB of Pandas DataFrame in Memory: {get_df_memory_size(pandas_df_spend)}')

Size in MB of Snowpark DataFrame in Memory: 0.0
Size in MB of Pandas DataFrame in Memory: 1.23


The Snowpark DataFrame consumes **zero** in memory MB - it is stored entirely within Snowflake. Whereas **the entire** Pandas DataFrame is stored in local memory

*Note: the ```.to_pandas()``` method is called to convert any Snowpark DataFrame to a Pandas DataFrame -- we'll use this method multiple times throughout the rest of the workshop*

### Creating new columns
We can add new columns to our Snowpark DataFrame using the ```.with_column()``` method.

Here, we are adding two new columns: *MONTH* and *YEAR* which represent the month and year an a campaign was run

In [6]:
snow_df_spend=snow_df_spend.with_column("YEAR", F.year(snow_df_spend["DATE"]))
snow_df_spend=snow_df_spend.with_column("MONTH", F.month(snow_df_spend["DATE"]))

snow_df_spend.show()

----------------------------------------------------------------------------------
|"CHANNEL"          |"DATE"      |"DAILYCOST"  |"DAILYLEADS"  |"YEAR"  |"MONTH"  |
----------------------------------------------------------------------------------
|Digital Not Set    |2021-01-01  |0            |353           |2021    |1        |
|Manually Entered   |2021-01-01  |0            |1483          |2021    |1        |
|Alloy              |2021-01-01  |0            |337           |2021    |1        |
|(not set)          |2021-01-01  |0            |29            |2021    |1        |
|Direct/Organic     |2021-01-01  |0            |910           |2021    |1        |
|Social             |2021-01-01  |22150        |700           |2021    |1        |
|SEM                |2021-01-01  |35396        |614           |2021    |1        |
|Display            |2021-01-01  |21068        |149           |2021    |1        |
|Radio              |2021-01-01  |0            |0             |2021    |1        |
|3rd

**YOUR TURN**: add a new column called *COST_PER_LEAD* which is the result of *DAILYCOST / DAILYLEADS*

*Expected output*:
```
---------------------------------------------------------------------------------
|"CHANNEL"          |"DATE"      |"DAILYCOST"  |"DAILYLEADS"  |"COST_PER_LEAD"  |
---------------------------------------------------------------------------------
|Digital Not Set    |2021-01-01  |0            |353           |0.000000         |
|Manually Entered   |2021-01-01  |0            |1483          |0.000000         |
|Alloy              |2021-01-01  |0            |337           |0.000000         |
|(not set)          |2021-01-01  |0            |29            |0.000000         |
|Direct/Organic     |2021-01-01  |0            |910           |0.000000         |
|Social             |2021-01-01  |22150        |700           |31.642857        |
|SEM                |2021-01-01  |35396        |614           |57.648208        |
|Display            |2021-01-01  |21068        |149           |141.395973       |
|Radio              |2021-01-01  |0            |0             |0.000000         |
|3rd Party Listing  |2021-01-01  |31446        |1174          |26.785349        |
---------------------------------------------------------------------------------
```

In [7]:
snow_df_spend=snow_df_spend.with_column("COST_PER_LEAD", F.div0(snow_df_spend["DAILYCOST"],snow_df_spend["DAILYLEADS"]))
snow_df_spend.select("CHANNEL","DATE","DAILYCOST","DAILYLEADS","COST_PER_LEAD").show()

---------------------------------------------------------------------------------
|"CHANNEL"          |"DATE"      |"DAILYCOST"  |"DAILYLEADS"  |"COST_PER_LEAD"  |
---------------------------------------------------------------------------------
|Digital Not Set    |2021-01-01  |0            |353           |0.000000         |
|Manually Entered   |2021-01-01  |0            |1483          |0.000000         |
|Alloy              |2021-01-01  |0            |337           |0.000000         |
|(not set)          |2021-01-01  |0            |29            |0.000000         |
|Direct/Organic     |2021-01-01  |0            |910           |0.000000         |
|Social             |2021-01-01  |22150        |700           |31.642857        |
|SEM                |2021-01-01  |35396        |614           |57.648208        |
|Display            |2021-01-01  |21068        |149           |141.395973       |
|Radio              |2021-01-01  |0            |0             |0.000000         |
|3rd Party Listi

We can also create new columns using *if-then-else* logic by using the ```.iff()``` method.

Suppose we want to flag a channel as either being focused on **DIGITAL** or something else. We can can do so by running the code block below

In [8]:
Digital_Channels=[
    'Digital Not Set', 'Social',
    'Email'
]

snow_df_spend=snow_df_spend.with_column("IS_DIGITAL", F.iff(snow_df_spend["CHANNEL"].isin(Digital_Channels), 1, 0))
snow_df_spend.select("CHANNEL","IS_DIGITAL").show(10)

------------------------------------
|"CHANNEL"          |"IS_DIGITAL"  |
------------------------------------
|Digital Not Set    |1             |
|Manually Entered   |0             |
|Alloy              |0             |
|(not set)          |0             |
|Direct/Organic     |0             |
|Social             |1             |
|SEM                |0             |
|Display            |0             |
|Radio              |0             |
|3rd Party Listing  |0             |
------------------------------------



Snowpark supports enhancing DataFrames in a number of different ways. The code block below demonstrates using the ```.when()``` method to add a column indicating the season in which an ad was run.

*Note: this is synonomous with a ```CASE-WHEN``` expression in SQL*

In [9]:
seasons=(
    F.when(snow_df_spend["MONTH"].isin(12, 1, 2),"winter")
    .when(snow_df_spend["MONTH"].isin(3, 4, 5),"spring")
    .when(snow_df_spend["MONTH"].isin(6, 7, 8),"summer")
    .when(snow_df_spend["MONTH"].isin(9, 10, 11),"fall")
)

snow_df_spend=snow_df_spend.with_column("SEASON", seasons)
snow_df_spend.select("CHANNEL","DATE","SEASON").show(12)


---------------------------------------------
|"CHANNEL"          |"DATE"      |"SEASON"  |
---------------------------------------------
|SEM                |2021-08-21  |summer    |
|3rd Party Listing  |2021-08-21  |summer    |
|Social             |2021-08-22  |summer    |
|SEM                |2021-08-22  |summer    |
|3rd Party Listing  |2021-08-22  |summer    |
|Manually Entered   |2021-08-22  |summer    |
|Display            |2021-08-22  |summer    |
|Direct/Organic     |2021-08-22  |summer    |
|Email              |2021-08-22  |summer    |
|Radio              |2021-08-22  |summer    |
|Alloy              |2021-08-22  |summer    |
|Digital Not Set    |2021-08-22  |summer    |
---------------------------------------------



**YOUR TURN**: suppose there was a reporting error in the winter season and the total cost of each ad was accidentally entered 10% too high.

Use the `.iff()` and `.with_column()` methods to overwrite the TOTAL_COST column so that is 10% lower for any ad run in the winter season.

_Expected output:_
```
   ----------------------------------------------
|"CHANNEL"          |"SEASON"  |"DAILYCOST"  |
----------------------------------------------
|Digital Not Set    |winter    |0.0          |
|Manually Entered   |winter    |0.0          |
|Alloy              |winter    |0.0          |
|(not set)          |winter    |0.0          |
|Direct/Organic     |winter    |0.0          |
|Social             |winter    |22150.0      |
|SEM                |winter    |35396.0      |
|Display            |winter    |21068.0      |
|Radio              |winter    |0.0          |
|3rd Party Listing  |winter    |31446.0      |
----------------------------------------------
```

In [10]:
snow_df_spend=snow_df_spend.with_column("DAILYCOST", F.iff(F.col("SEASON")=="season", F.col("DAILYCOST")*0.9, F.col("DAILYCOST")))
snow_df_spend.select("CHANNEL","SEASON","DAILYCOST").show(10)


----------------------------------------------
|"CHANNEL"          |"SEASON"  |"DAILYCOST"  |
----------------------------------------------
|Digital Not Set    |winter    |0.0          |
|Manually Entered   |winter    |0.0          |
|Alloy              |winter    |0.0          |
|(not set)          |winter    |0.0          |
|Direct/Organic     |winter    |0.0          |
|Social             |winter    |22150.0      |
|SEM                |winter    |35396.0      |
|Display            |winter    |21068.0      |
|Radio              |winter    |0.0          |
|3rd Party Listing  |winter    |31446.0      |
----------------------------------------------



### Aggregating Snowpark DataFrames
Now, that we have an idea for what our dataset looks like, let's use different Python and Snowpark to derive useful information from the raw data

First, we'll take a look at the total cost per ad channel

In [11]:
snow_df_spend_per_channel=(
    snow_df_spend
        .group_by("CHANNEL")
        .agg(F.sum("DAILYCOST").as_("TOTAL_COST"))
)

snow_df_spend_per_channel.show(50)


------------------------------------
|"CHANNEL"          |"TOTAL_COST"  |
------------------------------------
|SEM                |38708512.0    |
|3rd Party Listing  |17298512.0    |
|Social             |42966080.0    |
|Display            |16236784.0    |
|Offline Marketing  |106176.0      |
|Digital Not Set    |105968.0      |
|(not set)          |0.0           |
|Manually Entered   |0.0           |
|Alloy              |0.0           |
|Direct/Organic     |0.0           |
|Radio              |38752.0       |
|Email              |20816.0       |
|Spend Form         |649120.0      |
|OTT                |1840112.0     |
------------------------------------



Additionally, if we convert our Snowpark DataFrames to Pandas, we have the ability to create graphs and visualizations using any standard Python plotting library.

Here we are using the ```plotly.express``` library to create a bar chart of total cost per channel.

In [12]:
df=snow_df_spend_per_channel.to_pandas()
fig=px.bar(df, x="CHANNEL", y="TOTAL_COST")
fig.show()

##### Aggregating multiple columns
Just like with Pandas, Snowpark DataFrames support grouping and aggregating multiple columns.

This first code block creates a DataFrame which contains the total cost, average cost, and number of campaigns run for each channel

TIP: For a full list of functions, refer to the [docs](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/index).

In [13]:
# Stats per Month per Channel
snow_df_annual_spend_per_channel=(
    snow_df_spend
        .group_by(
            "YEAR",
            "CHANNEL"
        )
        .agg(
            F.sum("DAILYCOST").as_("TOTAL_COST"),
            F.avg("COST_PER_LEAD").as_("AVG_COST_PER_LEAD")
        )
        .sort("YEAR","CHANNEL")
)
snow_df_annual_spend_per_channel.show()

-------------------------------------------------------------------
|"YEAR"  |"CHANNEL"          |"TOTAL_COST"  |"AVG_COST_PER_LEAD"  |
-------------------------------------------------------------------
|2021    |(not set)          |0.0           |0E-12                |
|2021    |3rd Party Listing  |5403248.0     |18.555009095890      |
|2021    |Alloy              |0.0           |0E-12                |
|2021    |Digital Not Set    |1904.0        |0.019610652055       |
|2021    |Direct/Organic     |0.0           |0E-12                |
|2021    |Display            |2060288.0     |59.302212208219      |
|2021    |Email              |0.0           |0E-12                |
|2021    |Manually Entered   |0.0           |0E-12                |
|2021    |Radio              |0.0           |0E-12                |
|2021    |SEM                |6656976.0     |28.956007057534      |
-------------------------------------------------------------------



In [14]:
df=snow_df_annual_spend_per_channel.to_pandas()

fig=px.line(
    df,
    x="YEAR",
    y="TOTAL_COST",
    color="CHANNEL",
    markers=True
)

# Update the x-axis to show only integer year labels
fig.update_xaxes(dtick=1)

fig.show()

**YOUR TURN**: Create a DataFrame which shows the total cost per month per channel. Be sure to include the *SEASON* column in the *group by* expression
  
*Expected output*:
```
------------------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEASON"  |"CHANNEL"          |"TOTAL_COST"  |
------------------------------------------------------------------
|2021    |1        |winter    |Display            |337088.0      |
|2021    |1        |winter    |Radio              |0.0           |
|2021    |1        |winter    |Digital Not Set    |0.0           |
|2021    |1        |winter    |(not set)          |0.0           |
|2021    |1        |winter    |Manually Entered   |0.0           |
|2021    |1        |winter    |Direct/Organic     |0.0           |
|2021    |1        |winter    |Email              |0.0           |
|2021    |1        |winter    |3rd Party Listing  |503136.0      |
|2021    |1        |winter    |Alloy              |0.0           |
|2021    |1        |winter    |SEM                |566336.0      |
------------------------------------------------------------------
```

In [15]:
snow_df_monthly_spend_per_channel=(
    snow_df_spend
        .group_by(
            "YEAR",
            "MONTH",
            "SEASON",
            "CHANNEL"
        )
        .agg(
            F.sum("DAILYCOST").as_("TOTAL_COST"),
        )
        .sort("YEAR","MONTH")
)

snow_df_monthly_spend_per_channel.show()

------------------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEASON"  |"CHANNEL"          |"TOTAL_COST"  |
------------------------------------------------------------------
|2021    |1        |winter    |Display            |337088.0      |
|2021    |1        |winter    |Radio              |0.0           |
|2021    |1        |winter    |Digital Not Set    |0.0           |
|2021    |1        |winter    |(not set)          |0.0           |
|2021    |1        |winter    |Manually Entered   |0.0           |
|2021    |1        |winter    |Direct/Organic     |0.0           |
|2021    |1        |winter    |Email              |0.0           |
|2021    |1        |winter    |3rd Party Listing  |503136.0      |
|2021    |1        |winter    |Alloy              |0.0           |
|2021    |1        |winter    |SEM                |566336.0      |
------------------------------------------------------------------



### Filtering Snowpark DataFrames
By using the ```.filter()``` method, we can filter Snowpark DataFrames just like in Pandas.

Here we are filtering our data to only include the *SEM* channel

In [16]:
snow_df_monthly_spend_per_channel_SEM = snow_df_monthly_spend_per_channel.filter(F.col("CHANNEL") == "SEM")
snow_df_monthly_spend_per_channel_SEM.show()

----------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEASON"  |"CHANNEL"  |"TOTAL_COST"  |
----------------------------------------------------------
|2021    |1        |winter    |SEM        |566336.0      |
|2021    |2        |winter    |SEM        |469664.0      |
|2021    |3        |spring    |SEM        |555520.0      |
|2021    |4        |spring    |SEM        |575360.0      |
|2021    |5        |spring    |SEM        |598400.0      |
|2021    |6        |summer    |SEM        |607840.0      |
|2021    |7        |summer    |SEM        |637568.0      |
|2021    |8        |summer    |SEM        |594528.0      |
|2021    |9        |fall      |SEM        |530208.0      |
|2021    |10       |fall      |SEM        |519552.0      |
----------------------------------------------------------



**YOUR TURN**: Show months where *TOTAL_COST* value was less than $510,000

*Expected output:*
```
------------------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEASON"  |"CHANNEL"          |"TOTAL_COST"  |
------------------------------------------------------------------
|2021    |1        |winter    |3rd Party Listing  |503136.0      |
|2021    |1        |winter    |Display            |337088.0      |
|2021    |1        |winter    |Radio              |0.0           |
|2021    |1        |winter    |Direct/Organic     |0.0           |
|2021    |1        |winter    |Email              |0.0           |
|2021    |1        |winter    |Alloy              |0.0           |
|2021    |1        |winter    |Manually Entered   |0.0           |
|2021    |1        |winter    |(not set)          |0.0           |
|2021    |1        |winter    |Digital Not Set    |0.0           |
|2021    |1        |winter    |Social             |354400.0      |
------------------------------------------------------------------
```

In [17]:
# Filter our dataframe on total cost less than $510,000
dff=snow_df_monthly_spend_per_channel.filter(F.col("TOTAL_COST") < 510000)
dff.show()

------------------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEASON"  |"CHANNEL"          |"TOTAL_COST"  |
------------------------------------------------------------------
|2021    |1        |winter    |3rd Party Listing  |503136.0      |
|2021    |1        |winter    |Display            |337088.0      |
|2021    |1        |winter    |Radio              |0.0           |
|2021    |1        |winter    |Direct/Organic     |0.0           |
|2021    |1        |winter    |Email              |0.0           |
|2021    |1        |winter    |Alloy              |0.0           |
|2021    |1        |winter    |Manually Entered   |0.0           |
|2021    |1        |winter    |(not set)          |0.0           |
|2021    |1        |winter    |Digital Not Set    |0.0           |
|2021    |1        |winter    |Social             |354400.0      |
------------------------------------------------------------------



**YOUR TURN**: Show months where *TOTAL_COST* was less than $510,000 only for the *Display* channel

*Hint: multiple logical expressions can be chained together using the ```&``` and ```|``` operators*

*Expected output:*
```
 ----------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEASON"  |"CHANNEL"  |"TOTAL_COST"  |
----------------------------------------------------------
|2021    |1        |winter    |Display    |337088.0      |
|2021    |2        |winter    |Display    |248592.0      |
|2021    |3        |spring    |Display    |196736.0      |
|2021    |4        |spring    |Display    |184768.0      |
|2021    |5        |spring    |Display    |182848.0      |
|2021    |6        |summer    |Display    |163616.0      |
|2021    |7        |summer    |Display    |146064.0      |
|2021    |8        |summer    |Display    |147088.0      |
|2021    |9        |fall      |Display    |133792.0      |
|2021    |10       |fall      |Display    |108608.0      |
----------------------------------------------------------
```

In [18]:
# Place an additional filter on the channel column to look at Display Media only
dff=snow_df_monthly_spend_per_channel.filter(
    (F.col("TOTAL_COST") < 510000) & 
    (F.col("CHANNEL") == "Display")
)

dff.show()

----------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEASON"  |"CHANNEL"  |"TOTAL_COST"  |
----------------------------------------------------------
|2021    |1        |winter    |Display    |337088.0      |
|2021    |2        |winter    |Display    |248592.0      |
|2021    |3        |spring    |Display    |196736.0      |
|2021    |4        |spring    |Display    |184768.0      |
|2021    |5        |spring    |Display    |182848.0      |
|2021    |6        |summer    |Display    |163616.0      |
|2021    |7        |summer    |Display    |146064.0      |
|2021    |8        |summer    |Display    |147088.0      |
|2021    |9        |fall      |Display    |133792.0      |
|2021    |10       |fall      |Display    |108608.0      |
----------------------------------------------------------



**YOUR TURN**: Create a Snowpark Dataframe which shows the total cost per year but only for radio and offline marketing channels

_Expected output:_
```
-------------------------
|"YEAR"  |"TOTAL_COST"  |
-------------------------
|2023    |38752.0       |
|2021    |0.0           |
|2022    |106176.0      |
-------------------------
```

In [19]:
dff=(
    snow_df_monthly_spend_per_channel
        .filter(F.col("CHANNEL").isin(["Radio","Offline Marketing"]))
        .group_by("YEAR")
        .agg(F.sum("TOTAL_COST").as_("TOTAL_COST"))
)

dff.show()

-------------------------
|"YEAR"  |"TOTAL_COST"  |
-------------------------
|2023    |38752.0       |
|2021    |0.0           |
|2022    |106176.0      |
-------------------------



### Pivot on Channel

 Let's further transform the campaign spend data using the ```.pivot()``` and ```.sum()``` Snowpark functions. This transformation results in a DataFrame where **the sum of each row represents the total cost across all channels** per month.

 Pivoting our DataFrame this way enables us to join with the revenue table (which we will load in the next section) such that we will have our input features and target variable in a single table for model training.

 TIP: For a full list of functions, refer to the [docs](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/_autosummary/snowflake.snowpark.functions.html#module-snowflake.snowpark.functions).

In [20]:
snow_df_spend_per_month_pivot=(
    snow_df_monthly_spend_per_channel
        .pivot(
            pivot_col="CHANNEL",
            values=["Offline Marketing", "Direct/Organic", "Manually Entered", "3rd Party Listing", "Email", "(not set)", "Social", "OTT", "Digital Not Set", "Alloy", "Radio", "Spend Form", "Display", "SEM"]
        )
        .sum("TOTAL_COST")
        .sort("YEAR","MONTH")
    )

snow_df_spend_per_month_pivot.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEASON"  |"'Offline Marketing'"  |"'Direct/Organic'"  |"'Manually Entered'"  |"'3rd Party Listing'"  |"'Email'"  |"'(not set)'"  |"'Social'"  |"'OTT'"  |"'Digital Not Set'"  |"'Alloy'"  |"'Radio'"  |"'Spend Form'"  |"'Display'"  |"'SEM'"   |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2021    |1        |winter    |NULL                   |0.0                 |0.0                   |503136.0               |0.0        |0.0            |354400.0    |NULL     |0.0                  |0.0        |0.0

Notice that the names of our pivoted columns are lower cased and enclosed in single quotes - this will cause some problems later on when we use this data to train our regression model.

We will rename the columns below to make them more "model-friendly"

In [21]:
snow_df_spend_per_month_pivot = (
    snow_df_spend_per_month_pivot
        .with_column_renamed("'Offline Marketing'", "OFFLINE MARKETING")
        .with_column_renamed("'Direct/Organic'", "DIRECT/ORGANIC")
        .with_column_renamed("'Manually Entered'", "MANUALLY ENTERED")
        .with_column_renamed("'3rd Party Listing'", "3RD PARTY LISTING")
        .with_column_renamed("'Email'", "EMAIL")
        .with_column_renamed("'(not set)'", "(NOT SET)")
        .with_column_renamed("'Social'", "SOCIAL")
        .with_column_renamed("'OTT'", "OTT")
        .with_column_renamed("'Digital Not Set'", "DIGITAL NOT SET")
        .with_column_renamed("'Alloy'", "ALLOY")
        .with_column_renamed("'Radio'", "RADIO")
        .with_column_renamed("'Spend Form'", "SPEND FORM")
        .with_column_renamed("'Display'", "DISPLAY")
        .with_column_renamed("'SEM'", "SEM")
)


snow_df_spend_per_month_pivot.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEASON"  |"OFFLINE MARKETING"  |"DIRECT/ORGANIC"  |"MANUALLY ENTERED"  |"3RD PARTY LISTING"  |"EMAIL"  |"(NOT SET)"  |"SOCIAL"  |"OTT"  |"DIGITAL NOT SET"  |"ALLOY"  |"RADIO"  |"SPEND FORM"  |"DISPLAY"  |"SEM"     |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2021    |1        |winter    |NULL                 |0.0               |0.0                 |503136.0             |0.0      |0.0          |354400.0  |NULL   |0.0                |0.0      |0.0      |NULL          |337088.0   |566336.0  |
|2021    |2        |winter    |NULL                 

### Total Leads per Month

Now let's load the Leads table

**YOUR TURN**: complete the code block below to load a table from Snowflake called ```monthly_revenue```. Save this table as a Snowpark DataFrame named ```snow_df_monthly_revenue```


*Expected output*:
```
------------------------------
|"YEAR"  |"MONTH"  |"LEADS"  |
------------------------------
|2021    |1        |94158    |
|2021    |2        |83372    |
|2021    |3        |91857    |
|2021    |4        |82126    |
|2021    |5        |76766    |
|2021    |6        |68930    |
|2021    |7        |66214    |
|2021    |8        |59200    |
|2021    |9        |74441    |
|2021    |10       |84843    |
------------------------------

```

In [22]:
snow_df_monthly_revenue=session.table('LEADS_SUMMARY').sort("YEAR","MONTH")
snow_df_monthly_revenue.show()


------------------------------
|"YEAR"  |"MONTH"  |"LEADS"  |
------------------------------
|2021    |1        |94158    |
|2021    |2        |83372    |
|2021    |3        |91857    |
|2021    |4        |82126    |
|2021    |5        |76766    |
|2021    |6        |68930    |
|2021    |7        |66214    |
|2021    |8        |59200    |
|2021    |9        |74441    |
|2021    |10       |84843    |
------------------------------



### Join Total Spend and Total Revenue per Month

Joining Snowpark DataFrames works in the same way as Pandas or SQL tables.

Next let's **join this revenue data with the transformed campaign spend data** so that our input features (i.e. cost per channel) and target variable (i.e. revenue) can be loaded into a single table for model training. 

In [23]:
snow_df_monthly_spend_and_revenue=(
    snow_df_spend_per_month_pivot.join(snow_df_monthly_revenue, ["YEAR","MONTH"])
)




with session.query_history() as history2:
    snow_df_monthly_spend_and_revenue.show(20)

history2.queries

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEASON"  |"OFFLINE MARKETING"  |"DIRECT/ORGANIC"  |"MANUALLY ENTERED"  |"3RD PARTY LISTING"  |"EMAIL"  |"(NOT SET)"  |"SOCIAL"   |"OTT"  |"DIGITAL NOT SET"  |"ALLOY"  |"RADIO"  |"SPEND FORM"  |"DISPLAY"  |"SEM"      |"LEADS"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2021    |1        |winter    |NULL                 |0.0               |0.0                 |503136.0             |0.0      |0.0          |354400.0   |NULL   |0.0                |0.0      |0.0      |NULL          |337088.0   |566336.0   |94158    |
|202

[QueryRecord(query_id='01afdcff-0001-6e45-0003-b7d2000ca552', sql_text='SELECT  *  FROM ( SELECT  *  FROM (( SELECT "YEAR" AS "YEAR", "MONTH" AS "MONTH", "SEASON" AS "SEASON", "OFFLINE MARKETING" AS "OFFLINE MARKETING", "DIRECT/ORGANIC" AS "DIRECT/ORGANIC", "MANUALLY ENTERED" AS "MANUALLY ENTERED", "3RD PARTY LISTING" AS "3RD PARTY LISTING", "EMAIL" AS "EMAIL", "(NOT SET)" AS "(NOT SET)", "SOCIAL" AS "SOCIAL", "OTT" AS "OTT", "DIGITAL NOT SET" AS "DIGITAL NOT SET", "ALLOY" AS "ALLOY", "RADIO" AS "RADIO", "SPEND FORM" AS "SPEND FORM", "DISPLAY" AS "DISPLAY", "SEM" AS "SEM" FROM ( SELECT "YEAR", "MONTH", "SEASON", "\'Offline Marketing\'" AS "OFFLINE MARKETING", "\'Direct/Organic\'" AS "DIRECT/ORGANIC", "\'Manually Entered\'" AS "MANUALLY ENTERED", "\'3rd Party Listing\'" AS "3RD PARTY LISTING", "\'Email\'" AS "EMAIL", "\'(not set)\'" AS "(NOT SET)", "\'Social\'" AS "SOCIAL", "\'OTT\'" AS "OTT", "\'Digital Not Set\'" AS "DIGITAL NOT SET", "\'Alloy\'" AS "ALLOY", "\'Radio\'" AS "RADIO", "\

## >>>>>>>>>> *CHECKPOINT : PAUSE HERE BEFORE MOVING ON* <<<<<<<<<<

### Model Training in Snowflake 

#### Features and Target

At this point we are ready to perform the following actions to create features and target for model training.

* Replace nulls with zeroes
* Exclude columns we don't need for modeling - Channels with zero costs
* Save features into a Snowflake table called MARKETING_BUDGETS_LEADS

TIP: To see how to handle missing values in Snowpark Python, refer to this [blog](https://medium.com/snowflake/handling-missing-values-with-snowpark-for-python-part-1-4af4285d24e6).

In [24]:
inits=input("Enter your initials")

In [25]:

#Change all null values to zeroes
snow_df_spend_and_revenue_per_month_clean = snow_df_monthly_spend_and_revenue.fillna(0)


Input value type doesn't match the target column data type, this replacement was skipped. Column Name: "SEASON", Type: StringType(), Input Value: 0, Type: <class 'int'>


In [26]:


# Exclude year column we don't need for modeling
#snow_df_spend_and_revenue_per_month_clean = snow_df_spend_and_revenue_per_month_clean.drop(['YEAR'])

#Exclude campaigns with zero costs from the dataframe above
snow_df_spend_and_revenue_per_month_clean = snow_df_spend_and_revenue_per_month_clean.drop(col('DIRECT/ORGANIC'),col('MANUALLY ENTERED'),col('(NOT SET)'),col('ALLOY'))
print (snow_df_spend_and_revenue_per_month_clean.columns)

# Save features into a Snowflake table call MARKETING_BUDGETS_LEADS
snow_df_spend_and_revenue_per_month_clean.write.mode('overwrite').save_as_table(f'MARKETING_BUDGETS_LEADS_{inits}')
snow_df_spend_and_revenue_per_month_clean.show()

['YEAR', 'MONTH', 'SEASON', '"OFFLINE MARKETING"', '"3RD PARTY LISTING"', 'EMAIL', 'SOCIAL', 'OTT', '"DIGITAL NOT SET"', 'RADIO', '"SPEND FORM"', 'DISPLAY', 'SEM', 'LEADS']
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEASON"  |"OFFLINE MARKETING"  |"3RD PARTY LISTING"  |"EMAIL"  |"SOCIAL"  |"OTT"  |"DIGITAL NOT SET"  |"RADIO"  |"SPEND FORM"  |"DISPLAY"  |"SEM"     |"LEADS"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2021    |1        |winter    |0.0                  |503136.0             |0.0      |354400.0  |0.0    |0.0                |0.0      |0.0           |337088.0   |566336.0  |94158    |
|2021    |2        |winter    |0.0                  |444608.0             |0.0      |286384.0  

**YOUR TURN**: How would you save another "copy" of this dataframe to a table to do further analysis?

In [27]:
# Try it using the method above (save_as_table)
snow_df_spend_and_revenue_per_month_clean.write.mode("overwrite").save_as_table(f"MARKETING_BUDGETS_LEADS_ANALYSIS_{inits}")


#### Python function to train a Linear Regression model using scikit-learn

Let's create a Python function that uses **scikit-learn and other packages which are already included in** [Snowflake Anaconda channel](https://repo.anaconda.com/pkgs/snowflake/) and therefore available on the server-side when executing the Python function as a Stored Procedure running in Snowflake.

This function takes the following as parameters:

* _session_: Snowflake Session object.
* _features_table_: Name of the table that holds the features and target variable.
* _number_of_folds_: Number of cross validation folds used in GridSearchCV.
* _polynomial_features_degress_: PolynomialFeatures as a preprocessing step.
* _train_accuracy_threshold_: Accuracy thresholds for train dataset. This values is used to determine if the model should be saved.
* _test_accuracy_threshold_: Accuracy thresholds for test dataset. This values is used to determine if the model should be saved.
* _save_model_: Boolean that determines if the model should be saved provided the accuracy thresholds are met.

TIP: For large datasets, Snowflake offers [Snowpark-optimized Warehouses](https://docs.snowflake.com/en/user-guide/warehouses-snowpark-optimized.html) or in-memory storage



In [28]:

def train_leads_prediction_model(
    session: Session, 
    features_table: str, 
    numeric_features: list,
    categorical_features: list,
    number_of_folds: int,
    train_accuracy_threshold: float, 
    test_accuracy_threshold: float, 
    save_model: bool) -> str:
    
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler, OneHotEncoder
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split, GridSearchCV

    import os
    from joblib import dump

    # Load features
    df = session.table(features_table).to_pandas()

    # Preprocess the Numeric columns
    numeric_transformer = Pipeline(
        steps=[
            ('scaler', StandardScaler())
        ]
    )

    # one hot encode categorical columns
    categorical_transformer=Pipeline(
        steps=[
            ("ohe", OneHotEncoder(handle_unknown="ignore"))
        ]
    )

    # Combine the preprocessed step together using the Column Transformer module
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features)
        ]
    )

    # The next step is the integrate the features we just preprocessed with our Machine Learning algorithm to enable us to build a model
    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", LinearRegression())
        ]
    )

    # define dataframes for features (X) and target (y)
    X = df.drop('LEADS', axis = 1)
    y = df['LEADS']

    # Split dataset into training and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

    # Use GridSearch to find the best fitting model based on number_of_folds folds
    model = GridSearchCV(pipeline, param_grid={}, cv=number_of_folds)

    
    model.fit(X_train, y_train)
    
    train_r2_score = model.score(X_train, y_train)
    test_r2_score = model.score(X_test, y_test)

    # create a string of output text to view model results when training/testing
    train_accuracy = f"Train accuracy: {train_r2_score:.2%}; Train accuracy threshold: {train_accuracy_threshold:.2%}"
    test_accuracy = f"Test accuracy: {test_r2_score:.2%}; Test accuracy threshold: {test_accuracy_threshold:.2%}"

    if not save_model:
        save_state = f"Running in test mode...not saving model"
    else:
        if train_r2_score >= train_accuracy_threshold and test_r2_score >= test_accuracy_threshold:
            save_state = "Accuracy is acceptable! Saving model"

            # Upload trained model to a stage
            model_output_dir = '/tmp'
            model_file = os.path.join(model_output_dir, f'model_{inits}.joblib')
            dump(model, model_file)
            session.file.put(model_file, "@demo_models",overwrite=True)
        else:
            save_state = "Accuracy threshold violated...not saving"

    return (
        train_accuracy,
        test_accuracy,
        save_state
    )

#### Test Python function before deploying it as a Stored Procedure on Snowflake

To make sure our model works, we'll first call it before deploying to Snowflake

In [29]:
train_leads_prediction_model(
    session=session,
    features_table=f"MARKETING_BUDGETS_LEADS_{inits}",
    numeric_features = [ 'OFFLINE MARKETING', '3RD PARTY LISTING', 'EMAIL','SOCIAL', 'OTT', 'DIGITAL NOT SET', 'RADIO', 'SPEND FORM', 'DISPLAY','SEM'],
    categorical_features = ["SEASON"],
    number_of_folds=10,
    train_accuracy_threshold=0.85,
    test_accuracy_threshold=0.85,
    save_model=False
)

('Train accuracy: 97.60%; Train accuracy threshold: 85.00%',
 'Test accuracy: 89.22%; Test accuracy threshold: 85.00%',
 'Running in test mode...not saving model')

### Create Stored Procedure to deploy model training code on Snowflake

Assuming the testing is complete and we're satisfied with the model, let's **register the model training Python function as a Snowpark Python Stored Procedure** by supplying the packages (_snowflake-snowpark-python,scikit-learn, and joblib_) it will need and use during execution.

NOTE: Be sure to name your stored procedure below using your initials or another unique identifier since you are using the same database & schema!

In [30]:
# Be sure to name the stored procedure using your initials or another unique identifier such as "train_revenue_prediction_model_STR"
session.sproc.register(
    func=train_leads_prediction_model,
    name=f"train_leads_prediction_model_{inits}",
    packages=['snowflake-snowpark-python','scikit-learn','joblib'],
    is_permanent=True,
    stage_location="@demo_sprocs",
    replace=True)

### Execute Stored Procedure to train model and deploy it on Snowflake

Now we're ready to train the model and save it onto a Snowflake stage so let's set _save_model = True_ and run/execute the Stored Procedure using _session.call()_ function. 

NOTE: Use the same stored procedure name you used previously

In [31]:
# Make sure to update the stored procedure name to what you set above
cross_validaton_folds=10
numeric_features=[ 'OFFLINE MARKETING', '3RD PARTY LISTING', 'EMAIL','SOCIAL', 'OTT', 'DIGITAL NOT SET', 'RADIO', 'SPEND FORM', 'DISPLAY','SEM']
categorical_features=["SEASON"]
polynomial_features_degrees=2
train_accuracy_threshold=0.85
test_accuracy_threshold=0.85
save_model=True

session.call(
    f"train_leads_prediction_model_{inits}",
    f"MARKETING_BUDGETS_leads_{inits}",
    numeric_features,
    categorical_features,
    cross_validaton_folds,
    train_accuracy_threshold,
    test_accuracy_threshold,
    save_model
)

"('Train accuracy: 97.60%; Train accuracy threshold: 85.00%', 'Test accuracy: 89.22%; Test accuracy threshold: 85.00%', 'Accuracy is acceptable! Saving model')"

## >>>>>>>>>> *CHECKPOINT 2 : PAUSE HERE BEFORE MOVING ON* <<<<<<<<<<

### Create Scalar User-Defined Function (UDF) for inference

Now to deploy this model for inference, let's **create and register a Snowpark Python UDF and add the trained model as a dependency**. Once registered, getting new predictions is as simple as calling the function by passing in data.

Scalar UDFs operate on a single row / set of data points and are great for online inference

NOTE: Similar to before with the stored procedure, make sure to add your initials or a unique qualifier to the UDF name below

In [33]:
# Remember to change the UDF name to include your initials or a unique identifier
session.clear_imports()
session.clear_packages()

# Add trained model and Python packages from Snowflake Anaconda channel available on the server-side as UDF dependencies
session.add_import(f'@demo_models/model_{inits}.joblib.gz')
session.add_packages('pandas','joblib','scikit-learn==1.3.0')

@F.udf(name=f'predict_leads_{inits}',session=session,replace=True,is_permanent=True,stage_location='@demo_udfs')
def predict_roi(budget_allocations: list) -> float:

    import sys
    import pandas as pd
    from joblib import load

    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]
    
    model_file = import_dir + f'model_{inits}.joblib.gz'
    model = load(model_file)
            
    features = ['SEASON','OFFLINE MARKETING', '3RD PARTY LISTING', 'EMAIL','SOCIAL', 'OTT', 'DIGITAL NOT SET', 'RADIO', 'SPEND FORM', 'DISPLAY','SEM']
    df = pd.DataFrame([budget_allocations], columns=features)
    roi = abs(model.predict(df)[0])
    
    return roi

<div style='text-align: center'>
    <img src="assets/snowpark_python_udfs.png" alt="Snowpark" style="width: 75%;"/>
</div>

### Call Scalar User-Defined Function (UDF) for inference on new data

 Once the UDF is registered, getting new predictions is as simple as calling the _call_udf()_ Snowpark Python function and passing in new datapoints.

Let's create a SnowPark DataFrame with some sample data and call the UDF to get new predictions.

In [34]:
# define list of columns and convert to an arry
col_list = ["SEASON",'OFFLINE MARKETING', '3RD PARTY LISTING', 'EMAIL','SOCIAL', 'OTT', 'DIGITAL NOT SET', 'RADIO', 'SPEND FORM', 'DISPLAY','SEM']
col_array = F.array_construct(*col_list)

new_data=[
    ("winter", 0, 450000, 0, 2500000, 1000000,5000,15000,70000,800000,2250000)
]
test_df=session.create_dataframe(new_data, schema=col_list)


test_df.select(
   "SEASON","OFFLINE MARKETING","3RD PARTY LISTING","EMAIL","SOCIAL", "OTT","DIGITAL NOT SET","RADIO",  "SPEND FORM","DISPLAY","SEM",
  F.call_udf(f"predict_leads_{inits}", col_array).as_("PREDICTED_LEADS")
).show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SEASON"  |"OFFLINE MARKETING"  |"3RD PARTY LISTING"  |"EMAIL"  |"SOCIAL"  |"OTT"    |"DIGITAL NOT SET"  |"RADIO"  |"SPEND FORM"  |"DISPLAY"  |"SEM"    |"PREDICTED_LEADS"   |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|winter    |0                    |450000               |0        |2500000   |1000000  |5000               |15000    |70000         |800000     |2250000  |194833.09772812057  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



### Create Vectorized User-Defined Function (UDF) using Batch API for inference

Here we will leverage the Python UDF Batch API to create a **vectorized** UDF which takes a Pandas Dataframe as input. This means that each call to the UDF receives a set/batch of rows compared to a Scalar UDF which gets one row as input. 

First we will create a helper function _load_model()_ that uses **cachetools** to make sure we only load the model once followed by _batch_predict_roi()_ function that does the inference. 

Vectorized UDFs are great for offline inference in batch mode.

Advantages of using the Batch API over Scalar UDFs:

* The potential for better performance if your Python code operates efficiently on batches of rows
* Less transformation logic required if you are calling into libraries that operate on Pandas DataFrames or Pandas arrays

NOTE: Like before, remember to change the UDF function name to include your initials or a unique identifier below

In [35]:
from generate_batch_data_customized import generate_random_table

generate_random_table(
    session=session, 
    num_records=100000,
    table_name=f"random_model_data_{inits}"
)

generating random data...
saving random data...
100000 random records generated!


In [36]:
# Remember to update the UDF function name below similar to before
session.clear_imports()
session.clear_packages()

import cachetools
from snowflake.snowpark.types import PandasSeries, PandasDataFrame

# Add trained model and Python packages from Snowflake Anaconda channel available on the server-side as UDF dependencies
session.add_import(f'@demo_models/model_{inits}.joblib.gz')
session.add_packages('pandas','joblib','scikit-learn','cachetools')

@cachetools.cached(cache={})
def load_model(filename):
    import joblib
    import sys
    import os

    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]

    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = joblib.load(file)
            return m

@F.udf(name=f"batch_predict_leads_{inits}",session=session,replace=True,is_permanent=True,stage_location='@demo_udfs')
def batch_predict_roi(budget_allocations_df: PandasDataFrame[str, int, int, int, int, int, int, int, int, int, int]) -> PandasSeries[float]:

    budget_allocations_df.columns=["SEASON","OFFLINE MARKETING","3RD PARTY LISTING","EMAIL","SOCIAL", "OTT","DIGITAL NOT SET","RADIO",  "SPEND FORM","DISPLAY","SEM"]
    model = load_model(f'model_{inits}.joblib.gz')

    return abs(model.predict(budget_allocations_df))

### Call Vectorized User-Defined Function (UDF) using Batch API for inference on new data

When you use the Batch API:

* You do not need to change how you write queries using Python UDFs. All batching is handled by the UDF framework rather than your own code
* NOTE: As with the non-batch / scalar API, there is no guarantee of which instances of your handler code will see which batches of input

In [37]:


batch_df=session.table(f"random_model_data_{inits}")

batch_preds=batch_df.select(
    "SEASON","OFFLINE MARKETING","3RD PARTY LISTING","EMAIL","SOCIAL", "OTT","DIGITAL NOT SET","RADIO",  "SPEND FORM","DISPLAY","SEM", 
    F.call_udf(f"batch_predict_leads_{inits}",
    F.col("SEASON"),F.col("OFFLINE MARKETING"), F.col("3RD PARTY LISTING"), F.col("EMAIL"), F.col("Social"), F.col("OTT"), F.col("DIGITAL NOT SET"), F.col("RADIO"), F.col("SPEND FORM"), F.col("DISPLAY"),  F.col("SEM")).as_("PREDICTED_LEADS"))

batch_preds.show(10)
print(f"Num rows: {batch_preds.count()}")
print(f"Snowpark df size: {get_df_memory_size(batch_preds)}")
print(f"Pandas df size: {get_df_memory_size(batch_preds.to_pandas())}")


-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SEASON"  |"OFFLINE MARKETING"  |"3RD PARTY LISTING"  |"EMAIL"  |"SOCIAL"  |"OTT"    |"DIGITAL NOT SET"  |"RADIO"  |"SPEND FORM"  |"DISPLAY"  |"SEM"    |"PREDICTED_LEADS"   |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|fall      |3134764              |1912690              |2526157  |1106276   |3628782  |1817175            |780361   |1392436       |1329262    |2896805  |117031.3831848928   |
|winter    |315086               |658936               |154954   |1162857   |1100985  |1598660            |750933   |1415911       |725771     |1130153  |1777349.1978854868  |
|winter    |1903797              |1522010              |1479572  |1122925   |1829309  |557220             |1688678  |152

In [38]:
batch_preds.write.mode("overwrite").save_as_table(f"BATCH_PREDICT_LEADS_{inits}")

In [48]:
def get_new_predicted_leads(session: Session, new_allocations: dict, season: str) -> float:
    return (
        session
            .sql(
                f"""SELECT predict_leads_{inits}(array_construct(
                    '{season}',
                    {new_allocations['OFFLINE MARKETING']},
                    {new_allocations['3RD PARTY LISTING']},
                    {new_allocations['EMAIL']},
                    {new_allocations['SOCIAL']},
                    {new_allocations['OTT']},
                    {new_allocations['DIGITAL NOT SET']},
                    {new_allocations['RADIO']},
                    {new_allocations['SPEND FORM']},
                    {new_allocations['DISPLAY']},
                    {new_allocations['SEM']}
                ))"""
            )
            .to_pandas()
            .iloc[0, 0]
    )

In [47]:
snow_df1 = session.table("MARKETING_BUDGETS_LEADS_RE").filter(
    (F.col("YEAR") == 2023) & 
    (F.col("MONTH") <=9)
)
snow_df2 = snow_df1.unpivot("Budget", "Channel", ["OFFLINE MARKETING","3RD PARTY LISTING","EMAIL","SOCIAL", "OTT","DIGITAL NOT SET","RADIO", "SPEND FORM","DISPLAY","SEM"])

df1 = snow_df1.to_pandas()
last_month_leads=df1["LEADS"].iloc[-1]


print(last_month_leads)
snow_df2.show(100)

204286
-------------------------------------------------------------------------
|"YEAR"  |"MONTH"  |"SEASON"  |"LEADS"  |"CHANNEL"          |"BUDGET"   |
-------------------------------------------------------------------------
|2023    |1        |winter    |187388   |OFFLINE MARKETING  |0.0        |
|2023    |1        |winter    |187388   |3RD PARTY LISTING  |583392.0   |
|2023    |1        |winter    |187388   |EMAIL              |2496.0     |
|2023    |1        |winter    |187388   |SOCIAL             |2323808.0  |
|2023    |1        |winter    |187388   |OTT                |0.0        |
|2023    |1        |winter    |187388   |DIGITAL NOT SET    |32.0       |
|2023    |1        |winter    |187388   |RADIO              |0.0        |
|2023    |1        |winter    |187388   |SPEND FORM         |0.0        |
|2023    |1        |winter    |187388   |DISPLAY            |648688.0   |
|2023    |1        |winter    |187388   |SEM                |1528480.0  |
|2023    |2        |winter    |

In [45]:
import streamlit as st
import altair as alt

df=snow_df2.to_pandas()

# Create the stacked bar chart for BUDGET
bars = alt.Chart(df).mark_bar().encode(
    x='MONTH:O',
    y=alt.Y('BUDGET:Q', stack='zero'),
    color='CHANNEL:N',
    tooltip=['CHANNEL', 'BUDGET', 'LEADS']
)

# Create the line chart for LEADS
leads = df.groupby('MONTH').agg({'LEADS':'first'}).reset_index()  # get the unique LEADS value for each month
line = alt.Chart(leads).mark_line(color='black', point=True).encode(
    x='MONTH:O',
    y='LEADS:Q',
    tooltip='LEADS'
)

# Combine the bar chart and the line chart
chart = bars + line
chart.show()




ValueError: show() method requires the altair_viewer package. See http://github.com/altair-viz/altair_viewer

**Snowpark Stored Procedures vs User-Defined Functions**

_In general, if you're processing a large dataset in a way where each row/batch can be processed independently - UDFs are always better, because the processing is automatically parallelized/scaled across the warehouse. For example, if you already have a trained ML model, and you're doing inference using that model on billions of rows. In that case, each row/batch can be computed independently._

_If the use case requires the full dataset to be in-memory (e.g. ML training), then a stored procedure is the way to go. A stored procedure is just a Python program that runs on a single warehouse node. (With a UDF it's not possible to load the full dataset into memory because the processing is done in a streaming fashion, one batch at a time._

### Automate Model (re)training using Snowflake Tasks - Example Only

Here is an example of how to create a Snowflake (Serverless or User-managed) Task to automate (re)training of the model. For example, every hour.

In [ ]:
#create_model_training_task = """
#CREATE OR REPLACE TASK sp_hourly_model_training_
#   WAREHOUSE = 'YOUR_WH'
#   SCHEDULE  = '60 MINUTE'
#AS
#   CALL train_revenue_prediction_model_('MARKETING_BUDGETS_FEATURES',10,2,0.85,0.85,True)
#"""
#session.sql(create_model_training_task).collect()

#session.sql("alter task sp_hourly_model_training_ resume").collect()

### Other Snowpark Resources

Quick Start Guides

* [Getting Started With Snowpark for Python and Streamlit](https://quickstarts.snowflake.com/guide/getting_started_with_snowpark_for_python_streamlit/index.html?index=..%2F..index#0)

* [Getting Started With Snowpark Python](https://quickstarts.snowflake.com/guide/getting_started_with_snowpark_python/index.html?index=..%2F..index#0)

* [Machine Learning with Snowpark Python](https://quickstarts.snowflake.com/guide/machine_learning_with_snowpark_python/index.html?index=..%2F..index#0)

Videos: Snowpark | A Look Under The Hood 

* [Snowpark API](https://www.youtube.com/watch?v=Me2auWdhlKk)

* [Snowpark User-Defined Functions (UDFs)](https://www.youtube.com/watch?v=-W1aL8XwvkE)

[Blogs on Medium](https://medium.com/snowflake/search?q=Snowpark)

* [Deploy Custom UDFs Using GitHub Actions](https://medium.com/snowflake/deploying-custom-python-packages-from-github-to-snowflake-f0bb396480c7)

* [Snowpark For Python Open Source: How I Contributed And So Can You](https://medium.com/snowflake/snowpark-for-python-open-source-how-i-contributed-and-so-can-you-7eb4baac355f)

[Demos on GitHub](https://github.com/Snowflake-Labs/snowpark-python-demos)

[Snowpark for Python Developer Guide](https://docs.snowflake.com/en/developer-guide/snowpark/python/index.html)
